# Project
# Project Name:- Identification of User Query on Stack Overflow Using Semantic Search
### Semantic search seeks to improve search accuracy by understanding the searcher's intent.

## Objectives of Project:-

### 1.To identify most relevant questions to a query [text similarity]
### 2.Identify the matching tags and pick top relevant questions from stack overflow.
### 3.To identify top k solutions of the problem. (sentiment analysis of review content)
### 4.Take important information from comments also.
### 5. More will be added here.....bcoz DATA SPEAKS A LOT

# Query based on tags by NLU and Similar search by stack api

In [1]:
query = input("So what is your query?")

So what is your query?how to get all the keys of a dictionary in python


In [2]:
import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, KeywordsOptions ,SyntaxOptions, SyntaxOptionsTokens

credentials = {"api_key": "FQIQwKhvfCy1pjNJqOZ6ZH3M8YZI6on5t8Hr0Eyumep2",
               "api_url": "https://gateway.watsonplatform.net/natural-language-understanding/api"}


natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2018-11-16',
    iam_apikey=credentials['api_key'],
    url=credentials['api_url']
)

text_test = query

response = natural_language_understanding.analyze(
    text=text_test,
    features=Features(keywords=KeywordsOptions(),syntax=SyntaxOptions(
        sentences=True,
        tokens=SyntaxOptionsTokens(
          lemma=True,
          part_of_speech=True,
        )))).get_result()
# print(type(response))
# print(json.dumps(response, indent=2))

In [3]:
response

{'keywords': [{'count': 1,
   'relevance': 0.983223,
   'text': 'keys of a dictionary'},
  {'count': 1, 'relevance': 0.016777, 'text': 'python'}],
 'language': 'en',
 'syntax': {'sentences': [{'location': [0, 49],
    'text': 'how to get all the keys of a dictionary in python'}],
  'tokens': [{'lemma': 'how',
    'location': [0, 3],
    'part_of_speech': 'ADV',
    'text': 'how'},
   {'lemma': 'to', 'location': [4, 6], 'part_of_speech': 'PART', 'text': 'to'},
   {'lemma': 'get',
    'location': [7, 10],
    'part_of_speech': 'VERB',
    'text': 'get'},
   {'lemma': 'all',
    'location': [11, 14],
    'part_of_speech': 'DET',
    'text': 'all'},
   {'lemma': 'the',
    'location': [15, 18],
    'part_of_speech': 'DET',
    'text': 'the'},
   {'lemma': 'key',
    'location': [19, 23],
    'part_of_speech': 'NOUN',
    'text': 'keys'},
   {'lemma': 'of',
    'location': [24, 26],
    'part_of_speech': 'ADP',
    'text': 'of'},
   {'lemma': 'a', 'location': [27, 28], 'part_of_speech': 'DE

In [4]:
def relevance_list_fun(response_eta):
    relevance_list = [x['text'] for x in response_eta['keywords']]
    relevance_list = [x.split() for x in relevance_list]
    final_list_eta = []
    for i in relevance_list:
    #     print(i,"len is ",len(i))
        if len(i) > 1:
    #         print(i)
            for j in i:
                final_list_eta.append(j)
        else:
            final_list_eta.append(i[0])
    return final_list_eta

In [5]:
final_list = relevance_list_fun(response)

# is your question related to these?

In [6]:
print("Are all the below tags related to your query correct?")
for i in final_list:
    print(i)

agree = input()

Are all the below tags related to your query correct?
keys
of
a
dictionary
python
yes


In [7]:
if agree == 'no' or agree == 'NO' or agree == 'No' or agree == 'nO':
    print("oh shit the coder have'nt decided it yet")

## Let's get top 30 questions with are relevant to the above tags and also title.

In [8]:
api_url_1 = "https://api.stackexchange.com/2.2/similar?order=desc&sort=relevance&tagged="+";".join(final_list)+"&title="
api_url_2 = query+"&site=stackoverflow"
api_url = api_url_1 + api_url_2
api_url

'https://api.stackexchange.com/2.2/similar?order=desc&sort=relevance&tagged=keys;of;a;dictionary;python&title=how to get all the keys of a dictionary in python&site=stackoverflow'

In [9]:
# let's get top 30 questions
import requests
top_vote_questions = requests.get(api_url)
top_vote_questions = top_vote_questions.json()

#Please don't add other code in this cell because it will use api again and again while testing, Mohit

In [10]:
# print(type(top_vote_questions))
# print("\n")
# print(top_vote_questions.keys())
# print(type(top_vote_questions['items']))
# print(len(top_vote_questions['items']))
# print(top_vote_questions['items'][1].keys())

In [11]:
top_vote_questions['items'][15]

{'accepted_answer_id': 45776034,
 'answer_count': 1,
 'closed_date': 1503174946,
 'closed_reason': 'duplicate',
 'creation_date': 1503174685,
 'is_answered': True,
 'last_activity_date': 1503175579,
 'link': 'https://stackoverflow.com/questions/45776022/how-to-sort-the-keys-of-a-dictionary-in-python',
 'owner': {'accept_rate': 89,
  'display_name': 'Kurt Peek',
  'link': 'https://stackoverflow.com/users/995862/kurt-peek',
  'profile_image': 'https://www.gravatar.com/avatar/d5efb6c1054808f7a79aaefbc64cce76?s=128&d=identicon&r=PG',
  'reputation': 11889,
  'user_id': 995862,
  'user_type': 'registered'},
 'question_id': 45776022,
 'score': -1,
 'tags': ['python', 'algorithm', 'graph'],
 'title': 'How to sort the keys of a dictionary in Python?',
 'view_count': 57}

## Now let's make a dataframe of the above questions and analyse them

In [12]:
from pandas import DataFrame
df = DataFrame(top_vote_questions['items'])

In [13]:
df

,accepted_answer_id,answer_count,closed_date,closed_reason,creation_date,is_answered,last_activity_date,last_edit_date,link,owner,question_id,score,tags,title,view_count
0,32139872.0,3,NaN,NaN,1440158368,True,1440159426,1.440159e+09,https://stackoverflow.com/questions/32139770/e...,"{'reputation': 736, 'user_id': 1945881, 'user_...",32139770,-2,"[python, dictionary]",Editing keys of a dictionary in Python,61
1,NaN,3,NaN,NaN,1431462316,True,1431474464,1.495543e+09,https://stackoverflow.com/questions/30200760/g...,"{'reputation': 19, 'user_id': 4888700, 'user_t...",30200760,-2,"[python, dictionary]",get keyerror when trying to remap the keys of ...,174
2,48887727.0,4,NaN,NaN,1519137383,True,1519139193,NaN,https://stackoverflow.com/questions/48887674/c...,"{'reputation': 75, 'user_id': 5682776, 'user_t...",48887674,0,"[python, dictionary, nlp]",Convert a string to keys of a given dictionary...,57
3,NaN,2,NaN,NaN,1426654546,True,1426654813,NaN,https://stackoverflow.com/questions/29114417/t...,"{'reputation': 32947, 'user_id': 156458, 'user...",29114417,1,"[python, dictionary]",The keys of a dictionary are not in the order ...,37
4,45038247.0,3,NaN,NaN,1499784733,True,1556629028,1.499787e+09,https://stackoverflow.com/questions/45037977/g...,"{'reputation': 176, 'user_id': 3715819, 'user_...",45037977,5,"[python, python-3.x, dictionary, types]",Get the type of the key of a dictionary,5448
5,NaN,2,NaN,NaN,1499492319,True,1499590859,1.499494e+09,https://stackoverflow.com/questions/44982918/f...,"{'reputation': 101, 'user_id': 5030952, 'user_...",44982918,2,"[python-3.x, dictionary]",finding the keys of all the largest values in ...,240
6,NaN,2,NaN,NaN,1446407246,True,1547116768,1.530103e+09,https://stackoverflow.com/questions/33466450/p...,"{'reputation': 163, 'user_id': 5378920, 'user_...",33466450,8,"[python, json, key]",print All the keys of a json file in python,25048
7,NaN,1,NaN,NaN,1465308843,True,1465309855,1.465309e+09,https://stackoverflow.com/questions/37681820/h...,"{'reputation': 16, 'user_id': 6359012, 'user_t...",37681820,0,"[python, python-2.7, python-3.x, dictionary]",How to get key of a dictionary in a list or tu...,110
8,20001085.0,3,NaN,NaN,1384518308,True,1384521158,NaN,https://stackoverflow.com/questions/20001045/h...,"{'reputation': 507, 'user_id': 837438, 'user_t...",20001045,2,"[python, dictionary, permutation]",How to generate the keys of a dictionary using...,709
9,49693014.0,4,NaN,NaN,1523018021,True,1523020132,NaN,https://stackoverflow.com/questions/49692917/a...,"{'reputation': 616, 'user_id': 8470971, 'user_...",49692917,3,"[python, dictionary]",Altering all keys of a dictionary,50


# Things in my mind to be done after looking at this dataframe.
## 1st Phase(No statistics considered)

### 1.) Separate rows with closed_reason as duplicate, to find the question of which it is duplicate, find them and      merge those question in the df after 1st phase if they match a certain criteria.
### 2.) Remove rows with closed_reason as off-topic,etc except duplicate
### 3.)Remove questions with 0 answers.(if is_answered == False)
### 4.) What to do with Negative Scores/Downvotes?

In [14]:
# Getting df_duplicate to be handled later.
df_duplicate = df[df.closed_reason == 'duplicate']
print(len(df_duplicate),"Closed questions because of duplicacy")

2 Closed questions because of duplicacy


In [15]:
# Removal of closed questions
df = df[df.closed_reason != 'duplicate']
df = df[df.closed_reason != 'off-topic']
df = df[df.closed_reason != "unclear what you're asking"]
df = df[df.closed_reason != "too broad"]
df = df[df.closed_reason != "primarily opinion-based"]
len(df)

27

In [16]:
# Removal of question with 0 ansswers
df = df[df.is_answered == True]
len(df)

26

In [17]:
df

,accepted_answer_id,answer_count,closed_date,closed_reason,creation_date,is_answered,last_activity_date,last_edit_date,link,owner,question_id,score,tags,title,view_count
0,32139872.0,3,NaN,NaN,1440158368,True,1440159426,1.440159e+09,https://stackoverflow.com/questions/32139770/e...,"{'reputation': 736, 'user_id': 1945881, 'user_...",32139770,-2,"[python, dictionary]",Editing keys of a dictionary in Python,61
1,NaN,3,NaN,NaN,1431462316,True,1431474464,1.495543e+09,https://stackoverflow.com/questions/30200760/g...,"{'reputation': 19, 'user_id': 4888700, 'user_t...",30200760,-2,"[python, dictionary]",get keyerror when trying to remap the keys of ...,174
2,48887727.0,4,NaN,NaN,1519137383,True,1519139193,NaN,https://stackoverflow.com/questions/48887674/c...,"{'reputation': 75, 'user_id': 5682776, 'user_t...",48887674,0,"[python, dictionary, nlp]",Convert a string to keys of a given dictionary...,57
3,NaN,2,NaN,NaN,1426654546,True,1426654813,NaN,https://stackoverflow.com/questions/29114417/t...,"{'reputation': 32947, 'user_id': 156458, 'user...",29114417,1,"[python, dictionary]",The keys of a dictionary are not in the order ...,37
4,45038247.0,3,NaN,NaN,1499784733,True,1556629028,1.499787e+09,https://stackoverflow.com/questions/45037977/g...,"{'reputation': 176, 'user_id': 3715819, 'user_...",45037977,5,"[python, python-3.x, dictionary, types]",Get the type of the key of a dictionary,5448
5,NaN,2,NaN,NaN,1499492319,True,1499590859,1.499494e+09,https://stackoverflow.com/questions/44982918/f...,"{'reputation': 101, 'user_id': 5030952, 'user_...",44982918,2,"[python-3.x, dictionary]",finding the keys of all the largest values in ...,240
6,NaN,2,NaN,NaN,1446407246,True,1547116768,1.530103e+09,https://stackoverflow.com/questions/33466450/p...,"{'reputation': 163, 'user_id': 5378920, 'user_...",33466450,8,"[python, json, key]",print All the keys of a json file in python,25048
7,NaN,1,NaN,NaN,1465308843,True,1465309855,1.465309e+09,https://stackoverflow.com/questions/37681820/h...,"{'reputation': 16, 'user_id': 6359012, 'user_t...",37681820,0,"[python, python-2.7, python-3.x, dictionary]",How to get key of a dictionary in a list or tu...,110
8,20001085.0,3,NaN,NaN,1384518308,True,1384521158,NaN,https://stackoverflow.com/questions/20001045/h...,"{'reputation': 507, 'user_id': 837438, 'user_t...",20001045,2,"[python, dictionary, permutation]",How to generate the keys of a dictionary using...,709
9,49693014.0,4,NaN,NaN,1523018021,True,1523020132,NaN,https://stackoverflow.com/questions/49692917/a...,"{'reputation': 616, 'user_id': 8470971, 'user_...",49692917,3,"[python, dictionary]",Altering all keys of a dictionary,50


In [18]:
# Removal of closed_reason and closed_date columns and others as they are unnecessary now
del df['closed_reason']
del df['closed_date']
del df['creation_date']
del df['is_answered']
del df['last_activity_date']
del df['last_edit_date']

In [19]:
df

,accepted_answer_id,answer_count,link,owner,question_id,score,tags,title,view_count
0,32139872.0,3,https://stackoverflow.com/questions/32139770/e...,"{'reputation': 736, 'user_id': 1945881, 'user_...",32139770,-2,"[python, dictionary]",Editing keys of a dictionary in Python,61
1,NaN,3,https://stackoverflow.com/questions/30200760/g...,"{'reputation': 19, 'user_id': 4888700, 'user_t...",30200760,-2,"[python, dictionary]",get keyerror when trying to remap the keys of ...,174
2,48887727.0,4,https://stackoverflow.com/questions/48887674/c...,"{'reputation': 75, 'user_id': 5682776, 'user_t...",48887674,0,"[python, dictionary, nlp]",Convert a string to keys of a given dictionary...,57
3,NaN,2,https://stackoverflow.com/questions/29114417/t...,"{'reputation': 32947, 'user_id': 156458, 'user...",29114417,1,"[python, dictionary]",The keys of a dictionary are not in the order ...,37
4,45038247.0,3,https://stackoverflow.com/questions/45037977/g...,"{'reputation': 176, 'user_id': 3715819, 'user_...",45037977,5,"[python, python-3.x, dictionary, types]",Get the type of the key of a dictionary,5448
5,NaN,2,https://stackoverflow.com/questions/44982918/f...,"{'reputation': 101, 'user_id': 5030952, 'user_...",44982918,2,"[python-3.x, dictionary]",finding the keys of all the largest values in ...,240
6,NaN,2,https://stackoverflow.com/questions/33466450/p...,"{'reputation': 163, 'user_id': 5378920, 'user_...",33466450,8,"[python, json, key]",print All the keys of a json file in python,25048
7,NaN,1,https://stackoverflow.com/questions/37681820/h...,"{'reputation': 16, 'user_id': 6359012, 'user_t...",37681820,0,"[python, python-2.7, python-3.x, dictionary]",How to get key of a dictionary in a list or tu...,110
8,20001085.0,3,https://stackoverflow.com/questions/20001045/h...,"{'reputation': 507, 'user_id': 837438, 'user_t...",20001045,2,"[python, dictionary, permutation]",How to generate the keys of a dictionary using...,709
9,49693014.0,4,https://stackoverflow.com/questions/49692917/a...,"{'reputation': 616, 'user_id': 8470971, 'user_...",49692917,3,"[python, dictionary]",Altering all keys of a dictionary,50


## 2nd Phase(based on some statistics)
### 1.) Take top 25% questions on the basis of following:-
#### a.) No of  StackOverflow tags of question matching with query tags by NLU
#### b.) Text Similarity of the question title with query. (using tf-idf(nltk),intent classification,watson NLC,etc)
#### c.) Matching the Similarity of Verb of Query with Verb of question titles. (Though there is lot to consider in this and maybe b.) already covers this aspect)

#### d.) Sentiment Analysis of the comments to check if the question is nicely edited or not. If Positive comments than the question is nice but if negative comments than that question is not nice. (Keeping in mind that only appropriate comments can be there on stackoverflow as moderators remove unecessary comments?)

## a.) No of  StackOverflow tags of question matching with query tags by NLU

In [20]:
df['Tag_Match'] = df.apply(lambda row: len(set(row.tags).intersection(final_list)), axis = 1) 

In [21]:
df

,accepted_answer_id,answer_count,link,owner,question_id,score,tags,title,view_count,Tag_Match
0,32139872.0,3,https://stackoverflow.com/questions/32139770/e...,"{'reputation': 736, 'user_id': 1945881, 'user_...",32139770,-2,"[python, dictionary]",Editing keys of a dictionary in Python,61,2
1,NaN,3,https://stackoverflow.com/questions/30200760/g...,"{'reputation': 19, 'user_id': 4888700, 'user_t...",30200760,-2,"[python, dictionary]",get keyerror when trying to remap the keys of ...,174,2
2,48887727.0,4,https://stackoverflow.com/questions/48887674/c...,"{'reputation': 75, 'user_id': 5682776, 'user_t...",48887674,0,"[python, dictionary, nlp]",Convert a string to keys of a given dictionary...,57,2
3,NaN,2,https://stackoverflow.com/questions/29114417/t...,"{'reputation': 32947, 'user_id': 156458, 'user...",29114417,1,"[python, dictionary]",The keys of a dictionary are not in the order ...,37,2
4,45038247.0,3,https://stackoverflow.com/questions/45037977/g...,"{'reputation': 176, 'user_id': 3715819, 'user_...",45037977,5,"[python, python-3.x, dictionary, types]",Get the type of the key of a dictionary,5448,2
5,NaN,2,https://stackoverflow.com/questions/44982918/f...,"{'reputation': 101, 'user_id': 5030952, 'user_...",44982918,2,"[python-3.x, dictionary]",finding the keys of all the largest values in ...,240,1
6,NaN,2,https://stackoverflow.com/questions/33466450/p...,"{'reputation': 163, 'user_id': 5378920, 'user_...",33466450,8,"[python, json, key]",print All the keys of a json file in python,25048,1
7,NaN,1,https://stackoverflow.com/questions/37681820/h...,"{'reputation': 16, 'user_id': 6359012, 'user_t...",37681820,0,"[python, python-2.7, python-3.x, dictionary]",How to get key of a dictionary in a list or tu...,110,2
8,20001085.0,3,https://stackoverflow.com/questions/20001045/h...,"{'reputation': 507, 'user_id': 837438, 'user_t...",20001045,2,"[python, dictionary, permutation]",How to generate the keys of a dictionary using...,709,2
9,49693014.0,4,https://stackoverflow.com/questions/49692917/a...,"{'reputation': 616, 'user_id': 8470971, 'user_...",49692917,3,"[python, dictionary]",Altering all keys of a dictionary,50,2


In [22]:
# count = 1
# len_list=[]
# for i in top_vote_questions['items']:
#     if i['is_answered']:
#         print("Q No.",count," contains ",set(i['tags']).intersection(final_list),"so total ",len(set(i['tags']).intersection(final_list)))
#         len_list.append(len(set(i['tags']).intersection(final_list)))
#     count+=1
# print("\n")
# print("Total no of questions answered and will be further took into account is",len(len_list))
# # set(top_vote_questions['items'][1]['tags']).intersection(final_list)

# b.) Text Similarity of the question title with query. (using tf-idf(nltk),intent classification,watson NLC,etc)

### Try with TF-IDF NLTK

In [38]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

# nltk.download('punkt') # if necessary...

stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def cosine_sim(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]

print (cosine_sim(df.iloc[0].title, query))
print (cosine_sim(df.iloc[1].title, query))
print (cosine_sim(df.iloc[3].title, query))

0.7765145304745156
0.5797386715376658
0.4112070550676187


In [33]:
df['Similarity_index'] = df.apply(lambda row:cosine_sim(row.title,query) , axis = 1)

In [34]:
df

,accepted_answer_id,answer_count,link,owner,question_id,score,tags,title,view_count,Tag_Match,Similarity_index
0,32139872.0,3,https://stackoverflow.com/questions/32139770/e...,"{'reputation': 736, 'user_id': 1945881, 'user_...",32139770,-2,"[python, dictionary]",Editing keys of a dictionary in Python,61,2,0.776515
1,NaN,3,https://stackoverflow.com/questions/30200760/g...,"{'reputation': 19, 'user_id': 4888700, 'user_t...",30200760,-2,"[python, dictionary]",get keyerror when trying to remap the keys of ...,174,2,0.579739
2,48887727.0,4,https://stackoverflow.com/questions/48887674/c...,"{'reputation': 75, 'user_id': 5682776, 'user_t...",48887674,0,"[python, dictionary, nlp]",Convert a string to keys of a given dictionary...,57,2,0.579739
3,NaN,2,https://stackoverflow.com/questions/29114417/t...,"{'reputation': 32947, 'user_id': 156458, 'user...",29114417,1,"[python, dictionary]",The keys of a dictionary are not in the order ...,37,2,0.411207
4,45038247.0,3,https://stackoverflow.com/questions/45037977/g...,"{'reputation': 176, 'user_id': 3715819, 'user_...",45037977,5,"[python, python-3.x, dictionary, types]",Get the type of the key of a dictionary,5448,2,0.503103
5,NaN,2,https://stackoverflow.com/questions/44982918/f...,"{'reputation': 101, 'user_id': 5030952, 'user_...",44982918,2,"[python-3.x, dictionary]",finding the keys of all the largest values in ...,240,1,0.656973
6,NaN,2,https://stackoverflow.com/questions/33466450/p...,"{'reputation': 163, 'user_id': 5378920, 'user_...",33466450,8,"[python, json, key]",print All the keys of a json file in python,25048,1,0.356300
7,NaN,1,https://stackoverflow.com/questions/37681820/h...,"{'reputation': 16, 'user_id': 6359012, 'user_t...",37681820,0,"[python, python-2.7, python-3.x, dictionary]",How to get key of a dictionary in a list or tu...,110,2,0.411207
8,20001085.0,3,https://stackoverflow.com/questions/20001045/h...,"{'reputation': 507, 'user_id': 837438, 'user_t...",20001045,2,"[python, dictionary, permutation]",How to generate the keys of a dictionary using...,709,2,0.356300
9,49693014.0,4,https://stackoverflow.com/questions/49692917/a...,"{'reputation': 616, 'user_id': 8470971, 'user_...",49692917,3,"[python, dictionary]",Altering all keys of a dictionary,50,2,0.503103


# c.) Matching the Similarity of Verb of Query with Verb of question titles. (Though there is lot to consider in this and maybe b.) already covers this aspect)

In [24]:
def part_of_speech(response_eta):
    relevance_list = response_eta['syntax']['tokens']
    final_list_eta = []
    for i in relevance_list:
        if i['part_of_speech']== "VERB":
            final_list_eta.append(i['lemma'])
    return final_list_eta

In [25]:
verbs = part_of_speech(response)

In [26]:
# import nltk
from nltk.corpus import wordnet 
synonyms = []

threshold_value = 0.5

verb_synset = wordnet.synset(verbs[0]+".v.01")
for syn in wordnet.synsets(verbs[0]):
    if syn.pos() == 'v':
#         print(syn,"with similarity ",syn.wup_similarity(verb_synset))
        if syn.wup_similarity(verb_synset) > threshold_value:
            for l in syn.lemmas():
                synonyms.append(l.name()) 

print(set(synonyms))

{'acquire', 'get'}


## Applying NLU to get keywords of every question title

In [28]:

for i in top_vote_questions['items']:
    print(i['title'])
    response = natural_language_understanding.analyze(
    text=i['title'],
    features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=2),syntax=SyntaxOptions(
        sentences=True,
        tokens=SyntaxOptionsTokens(
          lemma=True,
          part_of_speech=True,
        )))).get_result()
    final_list = relevance_list_fun(response)
    print(final_list)
    print("\n")

Editing keys of a dictionary in Python
['keys', 'of', 'a', 'dictionary', 'Python']


get keyerror when trying to remap the keys of a dictionary in python
['keys', 'of', 'a', 'dictionary', 'keyerror']


Convert a string to keys of a given dictionary in python
['string', 'keys']


The keys of a dictionary are not in the order when they are added?
['keys', 'of', 'a', 'dictionary', 'order']


Get the type of the key of a dictionary
['type', 'of', 'the', 'key', 'of', 'a', 'dictionary']


finding the keys of all the largest values in python dictionary?
['largest', 'values', 'keys']


print All the keys of a json file in python
['keys', 'of', 'a', 'json', 'file', 'python']


How to get key of a dictionary in a list or tuple?
['key', 'of', 'a', 'dictionary', 'list']


How to generate the keys of a dictionary using permutations
['keys', 'of', 'a', 'dictionary', 'permutations']


Altering all keys of a dictionary
['keys', 'of', 'a', 'dictionary']


How to compare user input with the keys of a di

# Let's apply Statistical and other measure on the newly created columns created above.

In [29]:
# from statistics import mean,mode,median
# print(mean(len_list))
# print(mode(len_list))
# print(median(len_list))

In [30]:
# from numpy import percentile
# percentile(len_list,75)

In [31]:
# # just simple test
# list1 = [1,2,1,1,1,1,1]
# list2 = [1,2,2,3,2,2,3,3,3,4]
# print(percentile(list1,75))
# print(percentile(list2,75))